# Configurações

## Importações

In [30]:
import os
import sys

# Adicionar o diretório pai (src) ao path do Python
sys.path.append(os.path.abspath('../'))

import pandas as pd

from constants import numerical_features, categorical_features

## Definição do Dataframe

In [10]:
df_movies = pd.read_csv("../datasets/data_clean.csv")

# Engenharia de Features

In [31]:
# 2. Engenharia de Features
print("\nEngenharia de Features...")

# Feature 1: Ano de Lançamento
df_movies['release_year'] = pd.to_datetime(df_movies['release_date']).dt.year
print("   Criada feature 'release_year'.")

# Feature 2: Idade do Filme (anos desde o lançamento)
current_year = pd.Timestamp.now().year
df_movies['movie_age'] = current_year - df_movies['release_year']
print("   Criada feature 'movie_age'.")

# Feature 3: Converter 'adult' para inteiro (0 ou 1)
df_movies['is_adult'] = df_movies['adult'].astype(int)
print("   Convertida feature 'adult' para 'is_adult' (0 ou 1).")

# Feature 4: Número de Gêneros (simples, podemos expandir isso)
df_movies['num_genres'] = df_movies['genre_ids'].apply(len)
print("   Criada feature 'num_genres'.")


Engenharia de Features...
   Criada feature 'release_year'.
   Criada feature 'movie_age'.
   Convertida feature 'adult' para 'is_adult' (0 ou 1).
   Criada feature 'num_genres'.


In [32]:
# 3. Definição da Variável Alvo (Target Variable)
# Vamos definir um filme como 'altamente avaliado' se a média de votos for > 7.0 E
# se tiver mais de um certo número de votos para garantir a relevância.
min_vote_count_threshold = 50  # Exemplo: apenas filmes com mais de 50 votos

df_movies['is_highly_rated'] = ((df_movies['vote_average'] >= 7.0) & 
                                (df_movies['vote_count'] >= min_vote_count_threshold)).astype(int)

print(f"\n3. Definida variável alvo 'is_highly_rated' (1 se vote_average >= 7.0 e vote_count >= {min_vote_count_threshold}, 0 caso contrário).")
print(f"   Distribuição da variável alvo: \n{df_movies['is_highly_rated'].value_counts(normalize=True)}")


3. Definida variável alvo 'is_highly_rated' (1 se vote_average >= 7.0 e vote_count >= 50, 0 caso contrário).
   Distribuição da variável alvo: 
is_highly_rated
0    0.63
1    0.37
Name: proportion, dtype: float64


In [ ]:
# 4. Seleção das Features para o Modelo
# Vamos usar algumas features numéricas e uma categórica.


# Garantir que todas as colunas existem
for col in numerical_features + categorical_features:
    if col not in df_movies.columns:
        print(f"AVISO: Coluna '{col}' não encontrada no DataFrame. Verifique a engenharia de features.")
        numerical_features = [f for f in numerical_features if f != col]
        categorical_features = [f for f in categorical_features if f != col]

features = numerical_features + categorical_features
print(f"\n4. Features selecionadas para o modelo: {features}")

# 5. Tratamento de Outliers (Exemplo simplificado: remover filmes muito antigos se 'release_year' for muito baixo)
# df_movies = df_movies[df_movies['release_year'] > 1950]  # Exemplo, dependendo do dataset


4. Features selecionadas para o modelo: ['vote_count', 'popularity', 'release_year', 'movie_age', 'num_genres', 'original_language', 'is_adult']


# Gerar dataset

In [34]:
os.makedirs('../datasets', exist_ok=True)
df_movies.to_csv("../datasets/eda.csv")